# Read data

In [1]:
import pandas as pd
import os

In [2]:
path = "data/20_harder_MCQs.csv"
basename = os.path.basename(path)
basename = os.path.splitext(basename)[0]

mcq_df = pd.read_csv(path)
mcq_df.describe(include="all")

,question,options,correct_answer
count,20,20,20.000000
unique,18,18,NaN
top,What is the best weed control strategy on a farm?,"Use mechanical control methods, apply herbicid...",NaN
freq,2,2,NaN
mean,NaN,NaN,1.200000
std,NaN,NaN,1.056309
min,NaN,NaN,0.000000
25%,NaN,NaN,0.000000
50%,NaN,NaN,1.000000
75%,NaN,NaN,2.000000


In [3]:
mcq_df['question'].value_counts()

question
What is the best weed control strategy on a farm?                                                                          2
How do I make organic fertilizer from animal manure?                                                                       2
How do I implement integrated pest management (IPM) on a farm?                                                             1
How do I, in the shortest duration and cost effectively correct for nutrient deficiency in my farm?                        1
How do I calibrate a knapsack sprayer to ensure accurate pesticide application?                                            1
What is the right procedure to harvest onions?                                                                             1
How do I use animal waste management on a mixed crop-livestock farm?                                                       1
How should I wear PPE?                                                                                              

In [4]:
mcq_df = mcq_df.drop_duplicates()
mcq_df = mcq_df.reset_index(drop=True)
mcq_df['question_id'] = mcq_df.index + 1

mcq_df

,question,options,correct_answer,question_id
0,"How do I, in the shortest duration and cost ef...",Test soil for nutrient deficiency or perform v...,0,1
1,How do I implement integrated pest management ...,Plant early and with the right seeds to ensure...,0,2
2,How do I calibrate a knapsack sprayer to ensur...,Measure the sprayer output over a known time a...,0,3
3,What is the right procedure to harvest onions?,"Harvest during the dry season, at about 90-150...",0,4
4,How should I wear PPE?,"Put on overall/coveralls first, then gloves, m...",2,5
5,How do I use animal waste management on a mixe...,Collect animal waste and dump it directly onto...,2,6
6,How do I determine the seed rate required for ...,"Calculate the field area, determine the crop v...",3,7
7,Last year I experienced significant losses in ...,Harvest the crop when the leaves are dry. Thre...,3,8
8,How do I make organic fertilizer from animal m...,"Collect manure, compost it, apply the compost ...",2,9
9,What is the best weed control strategy on a farm?,"Use mechanical control methods, apply herbicid...",1,10


# Set up API keys

In [5]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [6]:
def mcq_prompt(question, answer_options, country):
    # convert options to list
    answer_options = answer_options.split("|")
    answer_options = "\n".join([f"{chr(ord('A') + i)}. {option}" for i, option in enumerate(answer_options)])
    return (f"You provide smallholder farmers in {country} with agricultural advice and information. "
            f"For the following multiple choice question, please select the correct answer(s) from options A to E and "
            f"provide a detailed rationale for your choice. There can be multiple correct answers.\n\n"
            f"Question: {question}\n\n"
            f"Answer options:\n{answer_options}\n\n"
            f"Your response must follow this format exactly:\n\n"
            f"Correct answer: [OPTION(S)]\n\n"
            f"Rationale: [Not longer than 10 sentences]")

In [7]:
import litellm

# set callbacks
litellm.success_callback = ["langfuse"]
litellm.failure_callback = ["langfuse"]

[2025-06-18 11:35:54 - httpx:1025 - INFO] HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"


In [8]:
country = "Kenya"

In [9]:
def get_llm_answers(df, model, country, **kwargs):
    ids = []
    prompts = []
    answers = []
    correct_answers = []

    for _, row in df.iterrows():
        question = row["question"]
        id = row["question_id"]
        answer_options = row["options"]
        correct_answer = chr(ord('A') + row["correct_answer"])
        prompt = mcq_prompt(question, answer_options, country)

        response = litellm.completion(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            **kwargs
        )

        # Extract the answer from the response
        answer = response["choices"][0]["message"]["content"]

        # Append the id, prompt, and answer to the lists
        ids.append(id)
        prompts.append(prompt)
        answers.append(answer)
        correct_answers.append(correct_answer)

    # Create a DataFrame from the lists
    results_df = pd.DataFrame({
        "question_id": ids,
        "prompt": prompts,
        "answer": answers,
        "correct_answer": correct_answers
    })

    # Save the DataFrame to a CSV file
    model_name = model.replace("/", "-").replace(".", "-")
    csv_path = os.path.join("results", f"{basename}-{model_name}.csv")
    print(f"Saving results to {csv_path}")
    results_df.to_csv(csv_path, index=False)

    return results_df

# OpenAI

In [10]:
gpt_4o_df = get_llm_answers(mcq_df, "openai/gpt-4o-2024-08-06", country)

Saving results to results/20_harder_MCQs-openai-gpt-4o-2024-08-06.csv


In [11]:
gpt_4o_df

,question_id,prompt,answer,correct_answer
0,1,You provide smallholder farmers in Kenya with ...,"Correct answer: A, C\n\nRationale: Option A ou...",A
1,2,You provide smallholder farmers in Kenya with ...,"Correct answer: A, B, D\n\nRationale: Integrat...",A
2,3,You provide smallholder farmers in Kenya with ...,"Correct answer: A, C\n\nRationale: Calibrating...",A
3,4,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Option A provi...,A
4,5,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: The correct se...,C
5,6,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: Effective anim...,C
6,7,You provide smallholder farmers in Kenya with ...,"Correct answer: A, B, D\n\nRationale: Determin...",D
7,8,You provide smallholder farmers in Kenya with ...,"Correct answer: B, D\n\nRationale: Options B a...",D
8,9,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: Option C provi...,C
9,10,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale: The best weed ...,B


In [12]:
o1_df = get_llm_answers(mcq_df, "openai/o1-2024-12-17", country, max_completion_tokens=25000, reasoning_effort="medium")

Saving results to results/20_harder_MCQs-openai-o1-2024-12-17.csv


In [13]:
o1_df

,question_id,prompt,answer,correct_answer
0,1,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Option A inclu...,A
1,2,You provide smallholder farmers in Kenya with ...,"Correct answer: A, B\n\nRationale: Option A co...",A
2,3,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Measuring the ...,A
3,4,You provide smallholder farmers in Kenya with ...,"Correct answer: [A, C]\n\nRationale: Harvestin...",A
4,5,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: When donning P...,C
5,6,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: Composting ani...,C
6,7,You provide smallholder farmers in Kenya with ...,Correct answer: [D]\n\nRationale: Determining ...,D
7,8,You provide smallholder farmers in Kenya with ...,Correct answer: D\n\nRationale: Harvesting at ...,D
8,9,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: Option C provi...,C
9,10,You provide smallholder farmers in Kenya with ...,Correct answer: [B]\n\nRationale: Identificati...,B


In [14]:
o3_df = get_llm_answers(mcq_df, "openai/o3-2025-04-16", country, max_completion_tokens=25000, reasoning_effort="medium")

Saving results to results/20_harder_MCQs-openai-o3-2025-04-16.csv


In [15]:
o3_df

,question_id,prompt,answer,correct_answer
0,1,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: A follows the ...,A
1,2,You provide smallholder farmers in Kenya with ...,"Correct answer: A, B\n\nRationale: IPM begins ...",A
2,3,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Effective cali...,A
3,4,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Harvesting oni...,A
4,5,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: PPE should be ...,C
5,6,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: Directly dumpi...,C
6,7,You provide smallholder farmers in Kenya with ...,Correct answer: D\n\nRationale: Determining se...,D
7,8,You provide smallholder farmers in Kenya with ...,Correct answer: D\n\nRationale: Proper loss‐re...,D
8,9,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: Proper organic...,C
9,10,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale: Effective weed...,B


In [16]:
o4_mini_df = get_llm_answers(mcq_df, "openai/o4-mini-2025-04-16", country, max_completion_tokens=25000, reasoning_effort="medium")

Saving results to results/20_harder_MCQs-openai-o4-mini-2025-04-16.csv


In [17]:
o4_mini_df

,question_id,prompt,answer,correct_answer
0,1,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: \n1. Rapid co...,A
1,2,You provide smallholder farmers in Kenya with ...,"Correct answer: A, B\n\nRationale: \n1. Optio...",A
2,3,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: \n1. Proper c...,A
3,4,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: \n1. Onions s...,A
4,5,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: \n1. You shou...,C
5,6,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: Composting ani...,C
6,7,You provide smallholder farmers in Kenya with ...,"Correct answer: B, D\n\nRationale:\n1. You mus...",D
7,8,You provide smallholder farmers in Kenya with ...,Correct answer: D\n\nRationale: \n1. Proper t...,D
8,9,You provide smallholder farmers in Kenya with ...,"Correct answer: A, C\n\nRationale: \n1. Both ...",C
9,10,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale: \n1. Effectiv...,B


# Anthropic

In [18]:
claude_3_5_df = get_llm_answers(mcq_df, "anthropic/claude-3-5-sonnet-20241022", country)

Saving results to results/20_harder_MCQs-anthropic-claude-3-5-sonnet-20241022.csv


In [19]:
claude_3_5_df

,question_id,prompt,answer,correct_answer
0,1,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Option A provi...,A
1,2,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Option A is th...,A
2,3,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Option A provi...,A
3,4,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Option A provi...,A
4,5,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: The correct se...,C
5,6,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: Option C repre...,C
6,7,You provide smallholder farmers in Kenya with ...,Correct answer: D\n\nRationale: The most compr...,D
7,8,You provide smallholder farmers in Kenya with ...,Correct answer: D\n\nRationale: Option D is th...,D
8,9,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: Option C is th...,C
9,10,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale: Option B repre...,B


In [20]:
claude_3_7_df = get_llm_answers(mcq_df, "anthropic/claude-3-7-sonnet-20250219", country, max_completion_tokens=25000, reasoning_effort="medium")

Saving results to results/20_harder_MCQs-anthropic-claude-3-7-sonnet-20250219.csv


In [21]:
claude_3_7_df

,question_id,prompt,answer,correct_answer
0,1,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Option A repre...,A
1,2,You provide smallholder farmers in Kenya with ...,"Correct answer: A, B\n\nRationale: Both option...",A
2,3,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Proper knapsac...,A
3,4,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Option A provi...,A
4,5,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: Option C provi...,C
5,6,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: Option C repre...,C
6,7,You provide smallholder farmers in Kenya with ...,Correct answer: D\n\nRationale: Determining th...,D
7,8,You provide smallholder farmers in Kenya with ...,Correct answer: D\n\nRationale: Option D provi...,D
8,9,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: Option C provi...,C
9,10,You provide smallholder farmers in Kenya with ...,Correct answer: [B]\n\nRationale: Option B rep...,B


Deepseek models don't support the `reasoning_effort` parameter.

In [22]:
deepseek_v3_df = get_llm_answers(mcq_df, "deepseek/deepseek-chat", country, max_completion_tokens=25000)

Saving results to results/20_harder_MCQs-deepseek-deepseek-chat.csv


In [23]:
deepseek_v3_df

,question_id,prompt,answer,correct_answer
0,1,You provide smallholder farmers in Kenya with ...,"Correct answer: A, B \n\nRationale: Both opti...",A
1,2,You provide smallholder farmers in Kenya with ...,"Correct answer: A, B, C, D \n\nRationale: All...",A
2,3,You provide smallholder farmers in Kenya with ...,Correct answer: [A]\n\nRationale: Calibrating ...,A
3,4,You provide smallholder farmers in Kenya with ...,Correct answer: A \n\nRationale: The correct ...,A
4,5,You provide smallholder farmers in Kenya with ...,Correct answer: C \n\nRationale: The correct ...,C
5,6,You provide smallholder farmers in Kenya with ...,Correct answer: [C] \n\nRationale: Composting...,C
6,7,You provide smallholder farmers in Kenya with ...,"Correct answer: [A, B, D]\n\nRationale: \n- Op...",D
7,8,You provide smallholder farmers in Kenya with ...,Correct answer: D\n\nRationale: Option D is th...,D
8,9,You provide smallholder farmers in Kenya with ...,"Correct answer: A, B, C \n\nRationale: All th...",C
9,10,You provide smallholder farmers in Kenya with ...,Correct answer: [B] \n\nRationale: The best w...,B


In [24]:
deepseek_r1_df = get_llm_answers(mcq_df, "deepseek/deepseek-reasoner", country, max_completion_tokens=25000)

Saving results to results/20_harder_MCQs-deepseek-deepseek-reasoner.csv


In [25]:
deepseek_r1_df

,question_id,prompt,answer,correct_answer
0,1,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Option A is co...,A
1,2,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Integrated Pes...,A
2,3,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Calibrating a ...,A
3,4,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Onions should ...,A
4,5,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: Option C provi...,C
5,6,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: Composting ani...,C
6,7,You provide smallholder farmers in Kenya with ...,Correct answer: D\n\nRationale: Determining se...,D
7,8,You provide smallholder farmers in Kenya with ...,Correct answer: D\n\nRationale: Option D is co...,D
8,9,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: \nMaking organ...,C
9,10,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale: For smallholde...,B


# Gemini

In [27]:
gemini_2_5_pro_df = get_llm_answers(mcq_df, "gemini/gemini-2.5-pro-preview-05-06", country, max_completion_tokens=25000, reasoning_effort="medium")

Saving results to results/20_harder_MCQs-gemini-gemini-2-5-pro-preview-05-06.csv


In [28]:
gemini_2_5_pro_df

,question_id,prompt,answer,correct_answer
0,1,You provide smallholder farmers in Kenya with ...,"Correct answer: [A, B]\n\nRationale:\nOption A...",A
1,2,You provide smallholder farmers in Kenya with ...,"Correct answer: [A, B]\n\nRationale: Option A ...",A
2,3,You provide smallholder farmers in Kenya with ...,Correct answer: [A]\n\nRationale: Option A out...,A
3,4,You provide smallholder farmers in Kenya with ...,"Correct answer: [A, C]\n\nRationale: Both opti...",A
4,5,You provide smallholder farmers in Kenya with ...,Correct answer: [C]\n\nRationale: Option C des...,C
5,6,You provide smallholder farmers in Kenya with ...,"Correct answer: A, C\n\nRationale: Both compos...",C
6,7,You provide smallholder farmers in Kenya with ...,"Correct answer: [A, B, D]\n\nRationale:\nOptio...",D
7,8,You provide smallholder farmers in Kenya with ...,"Correct answer: [B, D]\n\nRationale:\nOption D...",D
8,9,You provide smallholder farmers in Kenya with ...,"Correct answer: A, C\n\nRationale: Option C pr...",C
9,10,You provide smallholder farmers in Kenya with ...,"Correct answer: B, C\n\nRationale: Option B re...",B


# Aggregate results

In [29]:
basename

'20_harder_MCQs'

In [30]:
import glob

# Get a list of all CSV files in the results directory
csv_files = glob.glob(f"results/{basename}-*.csv")
# Print the list of CSV files
print(csv_files)

['results/20_harder_MCQs-openai-o3-2025-04-16.csv', 'results/20_harder_MCQs-deepseek-deepseek-reasoner.csv', 'results/20_harder_MCQs-gemini-gemini-2-5-pro-preview-05-06.csv', 'results/20_harder_MCQs-openai-o4-mini-2025-04-16.csv', 'results/20_harder_MCQs-openai-o1-2024-12-17.csv', 'results/20_harder_MCQs-openai-gpt-4o-2024-08-06.csv', 'results/20_harder_MCQs-deepseek-deepseek-chat.csv', 'results/20_harder_MCQs-anthropic-claude-3-5-sonnet-20241022.csv', 'results/20_harder_MCQs-anthropic-claude-3-7-sonnet-20250219.csv']


In [31]:
len(csv_files)

9

In [32]:
# read all csv files, extract the model name from the filename, and concatenate them into a single dataframe
dfs = []
for file in csv_files:
    model_name = "-".join(os.path.basename(file).split("-")[1:]).split(".")[0]
    df = pd.read_csv(file)
    df["model"] = model_name
    dfs.append(df)

# concatenate all dataframes into a single dataframe
all_df = pd.concat(dfs, ignore_index=True)

# drop the prompt column
all_df = all_df.drop(columns=["prompt"])

# add country column
all_df["country"] = country

In [33]:
all_df

,question_id,answer,correct_answer,model,country
0,1,Correct answer: A\n\nRationale: A follows the ...,A,openai-o3-2025-04-16,Kenya
1,2,"Correct answer: A, B\n\nRationale: IPM begins ...",A,openai-o3-2025-04-16,Kenya
2,3,Correct answer: A\n\nRationale: Effective cali...,A,openai-o3-2025-04-16,Kenya
3,4,Correct answer: A\n\nRationale: Harvesting oni...,A,openai-o3-2025-04-16,Kenya
4,5,Correct answer: C\n\nRationale: PPE should be ...,C,openai-o3-2025-04-16,Kenya
...,...,...,...,...,...
157,14,Correct answer: B\n\nRationale: Intercropping ...,B,anthropic-claude-3-7-sonnet-20250219,Kenya
158,15,Correct answer: B\n\nRationale: Option B provi...,B,anthropic-claude-3-7-sonnet-20250219,Kenya
159,16,Correct answer: B\n\nRationale: Creating a pro...,B,anthropic-claude-3-7-sonnet-20250219,Kenya
160,17,Correct answer: B\n\nRationale: Option B is th...,B,anthropic-claude-3-7-sonnet-20250219,Kenya


In [34]:
# add the questions by merging with the original dataframe
all_df = all_df.merge(mcq_df[["question_id", "question"]], on="question_id", how="left")

In [35]:
all_df

,question_id,answer,correct_answer,model,country,question
0,1,Correct answer: A\n\nRationale: A follows the ...,A,openai-o3-2025-04-16,Kenya,"How do I, in the shortest duration and cost ef..."
1,2,"Correct answer: A, B\n\nRationale: IPM begins ...",A,openai-o3-2025-04-16,Kenya,How do I implement integrated pest management ...
2,3,Correct answer: A\n\nRationale: Effective cali...,A,openai-o3-2025-04-16,Kenya,How do I calibrate a knapsack sprayer to ensur...
3,4,Correct answer: A\n\nRationale: Harvesting oni...,A,openai-o3-2025-04-16,Kenya,What is the right procedure to harvest onions?
4,5,Correct answer: C\n\nRationale: PPE should be ...,C,openai-o3-2025-04-16,Kenya,How should I wear PPE?
...,...,...,...,...,...,...
157,14,Correct answer: B\n\nRationale: Intercropping ...,B,anthropic-claude-3-7-sonnet-20250219,Kenya,Which are the best practices when intercroppin...
158,15,Correct answer: B\n\nRationale: Option B provi...,B,anthropic-claude-3-7-sonnet-20250219,Kenya,How do I compost crop residues to enhance soil...
159,16,Correct answer: B\n\nRationale: Creating a pro...,B,anthropic-claude-3-7-sonnet-20250219,Kenya,How do I create a simple farm budget for one s...
160,17,Correct answer: B\n\nRationale: Option B is th...,B,anthropic-claude-3-7-sonnet-20250219,Kenya,How do I apply pesticide to ensure safety and ...


In [36]:
# reorder the columns
all_df = all_df[["model", "question_id", "question", "answer", "correct_answer", "country"]]
all_df

,model,question_id,question,answer,correct_answer,country
0,openai-o3-2025-04-16,1,"How do I, in the shortest duration and cost ef...",Correct answer: A\n\nRationale: A follows the ...,A,Kenya
1,openai-o3-2025-04-16,2,How do I implement integrated pest management ...,"Correct answer: A, B\n\nRationale: IPM begins ...",A,Kenya
2,openai-o3-2025-04-16,3,How do I calibrate a knapsack sprayer to ensur...,Correct answer: A\n\nRationale: Effective cali...,A,Kenya
3,openai-o3-2025-04-16,4,What is the right procedure to harvest onions?,Correct answer: A\n\nRationale: Harvesting oni...,A,Kenya
4,openai-o3-2025-04-16,5,How should I wear PPE?,Correct answer: C\n\nRationale: PPE should be ...,C,Kenya
...,...,...,...,...,...,...
157,anthropic-claude-3-7-sonnet-20250219,14,Which are the best practices when intercroppin...,Correct answer: B\n\nRationale: Intercropping ...,B,Kenya
158,anthropic-claude-3-7-sonnet-20250219,15,How do I compost crop residues to enhance soil...,Correct answer: B\n\nRationale: Option B provi...,B,Kenya
159,anthropic-claude-3-7-sonnet-20250219,16,How do I create a simple farm budget for one s...,Correct answer: B\n\nRationale: Creating a pro...,B,Kenya
160,anthropic-claude-3-7-sonnet-20250219,17,How do I apply pesticide to ensure safety and ...,Correct answer: B\n\nRationale: Option B is th...,B,Kenya


In [37]:
# save the dataframe to a CSV file
all_df.to_csv(f"{basename}-all.csv", index=False)